# RuBERT



In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 
     |████████████████████████████████| 895 kB 44.5 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertForMaskedLM,BertTokenizer, pipeline
model=BertForMaskedLM.from_pretrained('sberbank-ai/ruBert-base')
tokenizer=BertTokenizer.from_pretrained('sberbank-ai/ruBert-base', do_lower_case=False)
unmasker = pipeline('fill-mask', model=model,tokenizer=tokenizer)

Downloading:   0%|          | 0.00/521 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

# Предложения

In [51]:
with open("sentences.txt") as file:
  sentences = file.read().splitlines()
  sentences = list(filter(None, sentences))

In [52]:
with open("masked_sentences.txt") as file:
  masked_sentences = file.read().splitlines()
  masked_sentences = list(filter(None, masked_sentences))

# Предсказывание

In [53]:
for i in range(len(masked_sentences)):
  try:
    output = f"{sentences[i]}\n"
    output += f"{masked_sentences[i]}\n"
    for i in unmasker(masked_sentences[i]):
      output += f"{i['sequence']}\n"
  except (TypeError):
    output = ""
  print(output, end="\n")



А денег у нас всё вре всё всего осталось сто рублей.
А денег у нас всё [MASK] всё всего осталось сто рублей.
А денег у нас всё - всё всего осталось сто рублей.
А денег у нас всё равно всё всего осталось сто рублей.
А денег у нас всё – всё всего осталось сто рублей.
А денег у нас всё, всё всего осталось сто рублей.
А денег у нас всё не всё всего осталось сто рублей.

Ну мы значит по-бырику начинаем сдавать назад.
Ну мы значит по-[MASK] начинаем сдавать назад.
Ну мы значит по - моему начинаем сдавать назад.
Ну мы значит по - любому начинаем сдавать назад.
Ну мы значит по - настоящему начинаем сдавать назад.
Ну мы значит по - хорошему начинаем сдавать назад.
Ну мы значит по - нашему начинаем сдавать назад.

Думаем если мы развернёмся это будет значит движение по однос по одностороннему движению так нельзя.
Думаем если мы развернёмся это будет значит движение по [MASK] по одностороннему движению так нельзя.
Думаем если мы развернёмся это будет значит движение по дороге по одностороннему д